In [1]:
import pandas as pd
from DataUnderstanding import DataUnderstanding

## DataFrame:

In [2]:
path = r'../data/insurance.csv'
df = pd.read_csv(
    filepath_or_buffer=path,
    decimal='.',
    sep=','
)

In [3]:
display(df.head())

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Código:

In [4]:
class HealthInsurance(DataUnderstanding):
    pass

if __name__ == '__main__':
    health_insurance = HealthInsurance()

## Output:

### Data Understanding:

In [5]:
display(df.head(3))

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523
2,28,male,33.00,3,no,southeast,4449.4620


In [6]:
display(df.tail(3))

,age,sex,bmi,children,smoker,region,charges
1335,18,female,36.85,0,no,southeast,1629.8335
1336,21,female,25.80,0,no,southwest,2007.9450
1337,61,female,29.07,0,yes,northwest,29141.3603


In [7]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [8]:
health_insurance.verificando_as_dimensões_do_dataframe(
    dataframe=df
)

Linhas: 1338 
Colunas: 7


In [9]:
health_insurance.descrição(
    dataframe=df,
    vertical=True
)

,count,mean,std,min,10%,15%,25%,50%,75%,85%,95%,99%,max
age,1338.0,39.21,14.05,18.00,19.00,22.00,27.00,39.00,51.00,56.00,62.00,64.00,64.00
bmi,1338.0,30.66,6.10,15.96,22.99,24.27,26.30,30.40,34.69,37.05,41.11,46.41,53.13
children,1338.0,1.09,1.21,0.00,0.00,0.00,0.00,1.00,2.00,2.00,3.00,5.00,5.00
charges,1338.0,13270.42,12110.01,1121.87,2346.53,3171.84,4740.29,9382.03,16639.91,24990.17,41181.83,48537.48,63770.43


In [10]:
health_insurance.descrição_object(
    dataframe=df,
    vertical=True
)

,count,unique,top,freq
sex,1338,2,male,676
smoker,1338,2,no,1064
region,1338,4,southeast,364


In [22]:
health_insurance.verificando_tipos(
    dataframe=df
)

,Tipos
age,int64
sex,object
bmi,float64
children,int64
smoker,object
region,object
charges,float64


In [23]:
health_insurance.verificando_valores_nulos(
    dataframe=df
)

,Quantidade,Porcentagem
age,0,0.00%
sex,0,0.00%
bmi,0,0.00%
children,0,0.00%
smoker,0,0.00%
region,0,0.00%
charges,0,0.00%


In [24]:
health_insurance.verificando_valores_duplicados(
    dataframe=df
)

,Quantidade,Porcentagem
age,1291,96.49%
sex,1336,99.85%
bmi,790,59.04%
children,1332,99.55%
smoker,1336,99.85%
region,1334,99.70%
charges,1,0.07%


In [17]:
colunas = ['sex', 'smoker', 'region',]

In [21]:
for coluna in colunas:
    print(f'{coluna}:')
    result = health_insurance.frequencia_da_repetição_dos_valores(
        dataframe=df,
        coluna=coluna
    )
    display(result)

sex:


,sex,count
0,male,676
1,female,662


smoker:


,smoker,count
0,no,1064
1,yes,274


region:


,region,count
0,southeast,364
1,southwest,325
2,northwest,325
3,northeast,324
